<a href="https://colab.research.google.com/github/zsj-jaz/Capstone-G4-ReelGood/blob/main/code/preprocess/%5BG4_ReelGood_Jaz%5DPreprocessing_ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import kagglehub
import shutil
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
dataset_path = '/content/drive/MyDrive/datasets/the-movies-dataset'

In [ ]:
metadata_df = pd.read_csv(dataset_path+'/cleaned_metadata.csv')

In [ ]:
ratings_df = pd.read_csv(dataset_path+'/ratings.csv')

In [ ]:
metadata_df.head()


,id,title,year,genres,first_three_actors,director,original_language,imdb_id
0,862,Toy Story,1995,"Animation, Comedy, Family","Tom Hanks, Tim Allen, Don Rickles",John Lasseter,en,tt0114709
1,8844,Jumanji,1995,"Adventure, Fantasy, Family","Robin Williams, Jonathan Hyde, Kirsten Dunst",Joe Johnston,en,tt0113497
2,15602,Grumpier Old Men,1995,"Romance, Comedy","Walter Matthau, Jack Lemmon, Ann-Margret",Howard Deutch,en,tt0113228
3,31357,Waiting to Exhale,1995,"Comedy, Drama, Romance","Whitney Houston, Angela Bassett, Loretta Devine",Forest Whitaker,en,tt0114885
4,11862,Father of the Bride Part II,1995,Comedy,"Steve Martin, Diane Keaton, Martin Short",Charles Shyer,en,tt0113041


In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [ ]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


filters ratings_df to keep only rows where movieId exists in metadata_df['id'].

In [ ]:
# Step 1: Remove duplicate ratings
duplicates = ratings_df[ratings_df.duplicated(subset=['userId', 'movieId'], keep=False)]
duplicate_count = duplicates.shape[0]
print(f"Number of duplicate ratings: {duplicate_count}")

# Step 2: Filter ratings for valid movies
filtered_ratings = ratings_df[ratings_df['movieId'].isin(metadata_df['id'])]

filtered_ratings.info()

Number of duplicate ratings: 0
<class 'pandas.core.frame.DataFrame'>
Index: 11428394 entries, 0 to 26024284
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 436.0 MB


In [ ]:

# Step 3: Filter users with at least 20 ratings
user_counts = filtered_ratings['userId'].value_counts()
valid_users = user_counts[user_counts >= 20].index
filtered_ratings = filtered_ratings[filtered_ratings['userId'].isin(valid_users)]


filtered_ratings.info()


<class 'pandas.core.frame.DataFrame'>
Index: 10212749 entries, 59 to 26024284
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 389.6 MB


In [ ]:
filtered_ratings.head()

,userId,movieId,rating,timestamp
59,4,223,4.0,1042668576
60,4,415,4.0,1042667925
61,4,648,4.0,1042674800
66,4,1422,4.0,1042674861
68,4,1597,3.0,1042674787


In [ ]:
# filtered_ratings.to_csv(dataset_path + "/cleaned_ratings.csv", index=False)

In [ ]:
import pandas as pd

# # Load the filtered ratings dataset
# filtered_ratings = pd.read_csv(dataset_path + "/cleaned_ratings.csv")

# Step 1: Sort by userId and timestamp
filtered_ratings = filtered_ratings.sort_values(by=['userId', 'timestamp'])

# Step 2: Select the last rating for each user (Test Set)
test_set = filtered_ratings.groupby('userId').tail(1)

# Step 3: Select the second last rating for each user (Validation Set)
remaining_ratings = filtered_ratings.drop(test_set.index)
val_set = remaining_ratings.groupby('userId').tail(1)

# Step 4: Select all other ratings as Train Set
train_set = remaining_ratings.drop(val_set.index)

# # Save the train, validation, and test sets
# train_set.to_csv(dataset_path + "/train_ratings.csv", index=False)
# val_set.to_csv(dataset_path + "/val_ratings.csv", index=False)
# test_set.to_csv(dataset_path + "/test_ratings.csv", index=False)

print(f"Train set size: {len(train_set)}")
print(f"Validation set size: {len(val_set)}")
print(f"Test set size: {len(test_set)}")

Train set size: 9972455
Validation set size: 120147
Test set size: 120147


In [ ]:
# Delete the old DataFrame and any other unnecessary variables
del ratings_df

# Delete any other large variables or DataFrames
del filtered_ratings

# Run garbage collection to free up memory
import gc
gc.collect()

import os
os.system('echo 3 > /proc/sys/vm/drop_caches')

512

In [ ]:
# Step 5: Construct the rating matrix only using the train set
train_matrix = train_set.pivot(index="movieId", columns="userId", values="rating")

train_matrix.head()


userId,4,7,8,9,11,12,15,16,20,22,...,270879,270881,270883,270885,270887,270891,270892,270893,270894,270896
movieId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,3.5


In [ ]:
train_matrix.shape


(7486, 120147)

In [ ]:
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz

# Convert to sparse matrix (CSR format)
sparse_train_matrix = csr_matrix(train_matrix.values)

# Save to a file
save_npz("train_matrix.npz", sparse_train_matrix)

print("Sparse matrix saved successfully!")

NameError: name 'train_matrix' is not defined

In [ ]:
# # Save the train matrix
# train_matrix.to_csv(dataset_path + "/train_matrix.csv")

# print("Train matrix constructed and saved!")


In [ ]:
def sparsity(matrix):
  total_elements = matrix.size
  non_null_entries = matrix.count().sum()  # sum of non-null entries in each column
  sparsity = (total_elements - non_null_entries) / total_elements * 100

  print(f"Total elements in matrix: {total_elements}")
  print(f"Non-null entries: {non_null_entries}")
  print(f"Sparsity: {sparsity:.2f}%")
sparsity(train_matrix)

Total elements in matrix: 899420442
Non-null entries: 9972455
Sparsity: 98.89%
